In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from functools import reduce
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct
from rapidfuzz import fuzz
import ftfy
import re


In [2]:
#First three datasets come from https://www.kaggle.com/datasets/stevenpeutz/misinformation-fake-news-text-dataset-79k

In [3]:
FN1 = pd.read_csv('DataSet_Misinfo_FAKE.csv', index_col=0)

In [4]:
FN1

,text
0,Donald Trump just couldn t wish all Americans ...
1,House Intelligence Committee Chairman Devin Nu...
2,"On Friday, it was revealed that former Milwauk..."
3,"On Christmas day, Donald Trump announced that ..."
4,Pope Francis used his annual Christmas Day mes...
...,...
44422,The USA wants to divide Syria.\r\n\r\nGreat Br...
44423,The Ukrainian coup d'etat cost the US nothing ...
44424,The European Parliament falsifies history by d...
44425,The European Parliament falsifies history by d...


In [5]:
FN1.head()

,text
0,Donald Trump just couldn t wish all Americans ...
1,House Intelligence Committee Chairman Devin Nu...
2,"On Friday, it was revealed that former Milwauk..."
3,"On Christmas day, Donald Trump announced that ..."
4,Pope Francis used his annual Christmas Day mes...


In [6]:
FN1.columns

Index(['text'], dtype='object')

In [7]:
#Adding a column to label it as real news or fake news as the datasets themselves were not labeled excpt for filename

In [8]:
FN1['real'] = 0

In [9]:
FN1

,text,real
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0
...,...,...
44422,The USA wants to divide Syria.\r\n\r\nGreat Br...,0
44423,The Ukrainian coup d'etat cost the US nothing ...,0
44424,The European Parliament falsifies history by d...,0
44425,The European Parliament falsifies history by d...,0


In [10]:
FN1['text'][0]

'Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s greeting like this despicable, petty, infantile gibberish? Only Trump! His lack of decency won t ev

In [11]:
TN1 = pd.read_csv('DataSet_Misinfo_TRUE.csv', index_col=0)

In [12]:
TN1.head()

,text
0,The head of a conservative Republican faction ...
1,Transgender people will be allowed for the fir...
2,The special counsel investigation of links bet...
3,Trump campaign adviser George Papadopoulos tol...
4,President Donald Trump called on the U.S. Post...


In [13]:
TN1.columns

Index(['text'], dtype='object')

In [14]:
#Again adding a column to the data as it was not labelled as real or fake within dataset

In [15]:
TN1['real'] = 1

In [16]:
TN1

,text,real
0,The head of a conservative Republican faction ...,1
1,Transgender people will be allowed for the fir...,1
2,The special counsel investigation of links bet...,1
3,Trump campaign adviser George Papadopoulos tol...,1
4,President Donald Trump called on the U.S. Post...,1
...,...,...
34970,Most conservatives who oppose marriage equalit...,1
34971,The freshman senator from Georgia quoted scrip...,1
34972,The State Department told the Republican Natio...,1
34973,"ADDIS ABABA, Ethiopia —President Obama convene...",1


In [17]:
TN1['text'][0]

'The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that support education, scientific researc

In [18]:
RusN = pd.read_csv('EXTRA_RussianPropagandaSubset.csv', index_col=0)

In [19]:
RusN

,text
0,Ukraine has put itself in a situation when ext...
1,Regardless who was behind the recent attack on...
2,"Pilsudski is a historical figure, who establis..."
3,Washington (and to a large degree Brussels) ar...
4,The beneficiary of the resolution of the Europ...
...,...
7364,The USA wants to divide Syria.\r\n\r\nGreat Br...
7365,The Ukrainian coup d'etat cost the US nothing ...
7366,The European Parliament falsifies history by d...
7367,The European Parliament falsifies history by d...


In [20]:
RusN.columns

Index(['text'], dtype='object')

In [21]:
#Russian propaganda extra dataset from Kaggle, again fake and needs labelled

In [22]:
RusN['real'] = 0

In [23]:
RusN

,text,real
0,Ukraine has put itself in a situation when ext...,0
1,Regardless who was behind the recent attack on...,0
2,"Pilsudski is a historical figure, who establis...",0
3,Washington (and to a large degree Brussels) ar...,0
4,The beneficiary of the resolution of the Europ...,0
...,...,...
7364,The USA wants to divide Syria.\r\n\r\nGreat Br...,0
7365,The Ukrainian coup d'etat cost the US nothing ...,0
7366,The European Parliament falsifies history by d...,0
7367,The European Parliament falsifies history by d...,0


In [24]:
RusN['text'][0]

'Ukraine has put itself in a situation when external forces dictate how it should solve its internal problems. Kyiv expected that its Western allies will put pressure on Russia in the Normandy Four format. However, the allies realised that their confrontation with Russia will cost them more. Emmanuel Macron and Angela Merkel supported Russia\'s position and put even bigger pressure on Ukraine than Vladimir Putin did. Eventually Zelenskyy found himself in solitude, because Russia, Germany, and France had nearly identical positions regarding the resolution of the civil conflict in Ukraine. Ukrainian politicians wrongfully remain confident that the West will always unconditionally support any Russophobic regime and that Ukraine can just carry out its foreign policy and security under the West\'s authority to enjoy its protection.\r\n\r\nUkraine could slam the door and declare that it no longer needs the Normandy format. However, it is the last remaining material evidence of Ukraine\'s uni

In [25]:
# Remaining datasets come from https://www.kaggle.com/datasets/emineyetm/fake-news-detection-datasets

In [26]:
FN2 = pd.read_csv('FAKE.csv')

In [27]:
FN2

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


In [28]:
FN2['real'] = 0

In [29]:
FN2

,title,text,subject,date,real
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0


In [30]:
FN2.isnull().values.any()

False

In [31]:
#Have to eliminate close duplicates

In [32]:
FN2['text'][0]

'Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s greeting like this despicable, petty, infantile gibberish? Only Trump! His lack of decency won t ev

In [33]:
TN2 = pd.read_csv('TRUE.csv')

In [34]:
TN2.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [35]:
TN2.columns

Index(['title', 'text', 'subject', 'date'], dtype='object')

In [36]:
#This dataset also does not have a label for fake or real news

In [37]:
TN2['real'] = 1

In [38]:
TN2

,title,text,subject,date,real
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1


In [39]:
#Concerning that the datasets seem to have to first articles as nearly identical. 

In [40]:
TN2['text'][0]

'WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that support educat

In [41]:
#Combining the first three datasets as they have similar columns, method was chosen for less lines of code

In [42]:
News_df = reduce(lambda  left,right: pd.merge(left,right,on=['text', 'real'],
                                            how='outer'), [FN1, TN1, RusN])


In [43]:
News_df.columns

Index(['text', 'real'], dtype='object')

In [44]:
#This checks the first three datasets for empty values. Surprisingly there are some. I will eliminate them

In [45]:
News_df.isnull().values.any()

True

In [46]:
null_counts = News_df.isnull().sum()
null_counts[null_counts > 0].sort_values(ascending=False)

text    32
dtype: int64

In [47]:
News_df.dropna(inplace = True)
News_df

,text,real
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0
...,...,...
85403,It is not the border between Norway and Russia...,0
85404,NATO soldiers in uniforms are scaring away Rus...,0
85405,Russia agrees with China's stance on US and We...,0
85406,\r\n\r\nTallinn is fighting furiously with its...,0


In [48]:
News_df.isnull().values.any()

False

In [49]:
#Inserting these columns for ease of the reduce method of joining dataframes. 

In [50]:
News_df['subject'] = np.nan
News_df['title'] = np.nan
News_df['date'] = np.nan

In [51]:
News_df

,text,real,subject,title,date
0,Donald Trump just couldn t wish all Americans ...,0,NaN,NaN,NaN
1,House Intelligence Committee Chairman Devin Nu...,0,NaN,NaN,NaN
2,"On Friday, it was revealed that former Milwauk...",0,NaN,NaN,NaN
3,"On Christmas day, Donald Trump announced that ...",0,NaN,NaN,NaN
4,Pope Francis used his annual Christmas Day mes...,0,NaN,NaN,NaN
...,...,...,...,...,...
85403,It is not the border between Norway and Russia...,0,NaN,NaN,NaN
85404,NATO soldiers in uniforms are scaring away Rus...,0,NaN,NaN,NaN
85405,Russia agrees with China's stance on US and We...,0,NaN,NaN,NaN
85406,\r\n\r\nTallinn is fighting furiously with its...,0,NaN,NaN,NaN


In [52]:
News_df.isnull().values.any()

True

In [53]:
News_df.isnull()

,text,real,subject,title,date
0,False,False,True,True,True
1,False,False,True,True,True
2,False,False,True,True,True
3,False,False,True,True,True
4,False,False,True,True,True
...,...,...,...,...,...
85403,False,False,True,True,True
85404,False,False,True,True,True
85405,False,False,True,True,True
85406,False,False,True,True,True


In [54]:
News_df = News_df.fillna('None')

In [55]:
News_df

,text,real,subject,title,date
0,Donald Trump just couldn t wish all Americans ...,0,None,None,None
1,House Intelligence Committee Chairman Devin Nu...,0,None,None,None
2,"On Friday, it was revealed that former Milwauk...",0,None,None,None
3,"On Christmas day, Donald Trump announced that ...",0,None,None,None
4,Pope Francis used his annual Christmas Day mes...,0,None,None,None
...,...,...,...,...,...
85403,It is not the border between Norway and Russia...,0,None,None,None
85404,NATO soldiers in uniforms are scaring away Rus...,0,None,None,None
85405,Russia agrees with China's stance on US and We...,0,None,None,None
85406,\r\n\r\nTallinn is fighting furiously with its...,0,None,None,None


In [56]:
#Combining all the data

In [57]:
News_df = reduce(lambda  left,right: pd.merge(left,right,on=['text', 'real', 'subject', 'title', 'date'],
                                            how='outer'), [News_df, TN2, FN2])

In [58]:
News_df

,text,real,subject,title,date
0,Donald Trump just couldn t wish all Americans ...,0,None,None,None
1,House Intelligence Committee Chairman Devin Nu...,0,None,None,None
2,"On Friday, it was revealed that former Milwauk...",0,None,None,None
3,"On Christmas day, Donald Trump announced that ...",0,None,None,None
4,Pope Francis used his annual Christmas Day mes...,0,None,None,None
...,...,...,...,...,...
130269,21st Century Wire says As 21WIRE reported earl...,0,Middle-east,McPain: John McCain Furious That Iran Treated ...,"January 16, 2016"
130270,21st Century Wire says It s a familiar theme. ...,0,Middle-east,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,"January 16, 2016"
130271,Patrick Henningsen 21st Century WireRemember ...,0,Middle-east,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,"January 15, 2016"
130272,21st Century Wire says Al Jazeera America will...,0,Middle-east,How to Blow $700 Million: Al Jazeera America F...,"January 14, 2016"


In [59]:
#Obviously there are duplicates, what might be a good cut off to eliminate the duplicates. Also are the datasets basically the
#same? Might be a limitation on Fake News training in general. 
#Partial was chosen as topics may be similar and reword certain ideas, but different news sources. 

In [60]:
fuzz.partial_ratio(FN2['text'][0],FN1['text'][0])

100.0

In [61]:
x = (len(News_df['text'])-1)

In [62]:
#Traditional fuzzy matching was taking too long and 
#I did not need to compare every letter to itself.
#Also I limited number of characters and increased chunksize of text for increased efficiency with little theoretical downside. 
#Thus Tdidf was already implemented in the finding of near duplicates
#code adapted from https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536 and
#https://bergvca.github.io/2017/10/14/super-fast-string-matching.html
#citation for ftfy: Robyn Speer. (2021). ftfy (Version 6.0).


In [63]:
def ngrams(string, n = 3):
    string = ftfy.fix_text(string)
    string = string.encode('ascii', errors='ignore').decode()
    string = string.lower()
    chars_to_remove = [')',')','.','|','[',']','{','}',"'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',',' ')
    string = string.replace('-', ' ')
    string = string.replace('\n', '')
    string = string.title()
    string = re.sub(' +',' ',string).strip()
    string = ' '+ string +' '
    #string = re.sub(r'[,-./]|\sBD',r'', string) (test this later)
    ngrams = zip(*[string[i:40] for i in range (n)])
    return [''.join(ngram) for ngram in ngrams]

In [64]:
News_arts = News_df['text'].unique()
vectorizer = TfidfVectorizer(min_df=1, analyzer = ngrams)
tf_idf_matrix = vectorizer.fit_transform(News_arts)

In [65]:
#print(tf_idf_matrix)

In [66]:
def awesome_cossim_top(A, B, ntop, lower_bound=0):
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)
    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)
    return csr_matrix((data,indices,indptr),shape=(M,N))

In [67]:
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(),1000,.9)


In [68]:
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})

In [69]:
#Nothing below 1.0 seems to be an exact match, lets look at it sorted

In [70]:
matches_df = get_matches_df(matches, News_df['text'], top=1000)
matches_df = matches_df[matches_df['similairity'] < 0.99999]
matches_df.sample(20)

,left_side,right_side,similairity
737,Former reality show star Donald Trump has alwa...,(Reuters) - Six U.S. Senate Democrats on Thurs...,0.943554
245,White House Chief of Staff John Kelly is a Fou...,-Onions -Red peppers \nNotice they are all hea...,0.978108
248,White House Chief of Staff John Kelly is a Fou...,Abby Martin Exposes What Hillary Clinton Reall...,0.939113
241,White House Chief of Staff John Kelly is a Fou...,DUBAI (Reuters) - Iran said on Wednesday it ha...,0.978108
246,White House Chief of Staff John Kelly is a Fou...,Countering Hillary’s Coup With a Counter Coup ...,0.978108
243,White House Chief of Staff John Kelly is a Fou...,BNI Store Nov 6 2016 Like a good little sharia...,0.978108
72,Former reality show star Donald Trump just can...,Former reality show star Donald Trump has alwa...,0.943554
898,Donald Trump s son-in-law Jared Kushner has fo...,"Jared Kushner, Donald Trump s son-in-law, has ...",0.934720
6,House Intelligence Committee Chairman Devin Nu...,"The private investigator, Robert Mueller, had ...",0.952208
738,Former reality show star Donald Trump has alwa...,Former reality show star Donald Trump just can...,0.943554


In [71]:
#It seems that we can reasonably assume that less than 1.0 are not duplicates

In [72]:
matches_df.sort_values(['similairity'], ascending=False).head(20)


,left_side,right_side,similairity
540,Milwaukee County Sheriff David Clarke seems li...,NATO intends to turn the entire country of Mon...,0.989785
539,Milwaukee County Sheriff David Clarke seems li...,Brussels and Washington accuse Russia of inter...,0.989785
247,White House Chief of Staff John Kelly is a Fou...,"A Birmingham, Alabama ABC affiliate sought out...",0.978108
241,White House Chief of Staff John Kelly is a Fou...,DUBAI (Reuters) - Iran said on Wednesday it ha...,0.978108
246,White House Chief of Staff John Kelly is a Fou...,Countering Hillary’s Coup With a Counter Coup ...,0.978108
245,White House Chief of Staff John Kelly is a Fou...,-Onions -Red peppers \nNotice they are all hea...,0.978108
244,White House Chief of Staff John Kelly is a Fou...,By LAURA MOWAT \nLocals are fuming because the...,0.978108
243,White House Chief of Staff John Kelly is a Fou...,BNI Store Nov 6 2016 Like a good little sharia...,0.978108
242,White House Chief of Staff John Kelly is a Fou...,The U.S. Justice Department on Monday said it ...,0.978108
5,House Intelligence Committee Chairman Devin Nu...,"If this keeps up, the Democrats will never win...",0.972838


In [73]:
#I am inclined for simplicity sake to remove all with a similarity of 1.0, but lets look at these first 5
#There are 226 'exact matches' much less than I was fearful of at the beginning
#But I have to remember that this is a sample of the dataset. And that this is potentially 25% of my data.
# Lets run the original 

In [74]:
matches_df = get_matches_df(matches, News_df['text'], top=1000)
matches_df = matches_df[matches_df['similairity'] ==1.0]
matches_df.sample(20)

,left_side,right_side,similairity
234,"Pearson, an educational publishing company, is...","Pearson, an educational publishing company, is...",1.0
766,It s certainly no secret that Trump struggles ...,It s certainly no secret that Trump struggles ...,1.0
759,Donald Trump doesn t read that s a well-know...,Donald Trump doesn t read that s a well-know...,1.0
792,"The NRA has a new favorite toy, but there are ...","The NRA has a new favorite toy, but there are ...",1.0
929,"Donald Trump Jr., Paul Manafort, Jared Kushner...","Donald Trump Jr., Paul Manafort, Jared Kushner...",1.0
453,This is what happens when racists don t get th...,This is what happens when racists don t get th...,1.0
712,"Wednesday morning, Donald Trump started his da...","Wednesday morning, Donald Trump started his da...",1.0
275,Donald Trump s administration is probably the ...,Donald Trump s administration is probably the ...,1.0
363,"Wow, it s like a president can t even get a ...",WASHINGTON (Reuters) - President Donald Trump ...,1.0
990,Star Trek legend George Takei gloriously shred...,Star Trek legend George Takei gloriously shred...,1.0


In [75]:
matches_df.head(10)

,left_side,right_side,similairity
9,"On Christmas day, Donald Trump announced that ...","On Christmas day, Donald Trump announced that ...",1.0
13,In the wake of yet another court decision that...,In the wake of yet another court decision that...,1.0
14,Many people have raised the alarm regarding th...,Many people have raised the alarm regarding th...,1.0
25,Senate Majority Whip John Cornyn (R-TX) though...,Senate Majority Whip John Cornyn (R-TX) though...,1.0
27,"In this #METOO moment, many powerful men are b...","In this #METOO moment, many powerful men are b...",1.0
33,Judge Jeanine Pirro has continued her screamy...,CHICAGO (Reuters) - Agriculture leaders includ...,1.0
34,Judge Jeanine Pirro has continued her screamy...,Judge Jeanine Pirro has continued her screamy...,1.0
35,Donald Trump held a rally for Alabama Senate c...,Donald Trump held a rally for Alabama Senate c...,1.0
45,Arizona Republican Senator Jeff Flake has neve...,Arizona Republican Senator Jeff Flake has neve...,1.0
64,"While most of us were sleeping, Donald Trump s...","While most of us were sleeping, Donald Trump s...",1.0


In [76]:
matches_df.tail(10)

,left_side,right_side,similairity
962,Republicans and Democrats are upset over Attor...,Republicans and Democrats are upset over Attor...,1.0
964,"As you are probably aware, Donald Trump loves ...","As you are probably aware, Donald Trump decide...",1.0
965,"As you are probably aware, Donald Trump loves ...","As you are probably aware, Donald Trump loves ...",1.0
980,For someone that ran on the Drain the Swamp ...,For someone that ran on the Drain the Swamp ...,1.0
983,Trump s numerous attacks on former FBI Directo...,Trump s numerous attacks on former FBI Directo...,1.0
984,Donald Trump just refuses to admit that most A...,Donald Trump just refuses to admit that most A...,1.0
988,Donald Trump and his vice-president are eager ...,Donald Trump and his vice-president are eager ...,1.0
990,Star Trek legend George Takei gloriously shred...,Star Trek legend George Takei gloriously shred...,1.0
993,"By now, everyone is used to the rank ignorance...","By now, everyone is used to the rank ignorance...",1.0
994,We all know that the GOP is the party of anti-...,We all know that the GOP is the party of anti-...,1.0


In [77]:
#It seems reasonable to assume that anything identified as similarity of 1.0 is indeed a duplicate. 
#Now to remove them. 
#I have to be careful as these were a sample of my datasets. 
#Let's confirm I don't have more than 8000 duplicates

In [78]:
matches_df = get_matches_df(matches, News_df['text'], top=8000)
matches_df = matches_df[matches_df['similairity'] ==1.0]
matches_df.reset_index(inplace = True)

In [79]:
if matches_df['left_side'][0] == News_df['text'][0]:
    print(True)

In [80]:
#2278 out of 130274 are duplicates, let's check percentage

In [81]:
2278/130274

0.01748622134884935

In [82]:
#Less than 2 percent, that is perfectly reasonable given that we are working with a few data sets. 
#Let's remove these from News_df

In [83]:
# just checking to make sure that matches_df retained the entire article

In [84]:
matches_df['left_side'][0]

'On Christmas day, Donald Trump announced that he would  be back to work  the following day, but he is golfing for the fourth day in a row. The former reality show star blasted former President Barack Obama for playing golf and now Trump is on track to outpace the number of golf games his predecessor played.Updated my tracker of Trump s appearances at Trump properties.71 rounds of golf including today s. At this pace, he ll pass Obama s first-term total by July 24 next year. https://t.co/Fg7VacxRtJ pic.twitter.com/5gEMcjQTbH  Philip Bump (@pbump) December 29, 2017 That makes what a Washington Post reporter discovered on Trump s website really weird, but everything about this administration is bizarre AF. The coding contained a reference to Obama and golf:  Unlike Obama, we are working to fix the problem   and not on the golf course.  However, the coding wasn t done correctly.The website of Donald Trump, who has spent several days in a row at the golf course, is coded to serve up the fo

In [85]:
fuzz.ratio(News_df['text'][0],matches_df['left_side'][0])

42.24457384859714

In [86]:
News_df_s = News_df.sample(n = 1000)
News_df_s.reset_index(inplace = True)


In [87]:
x = len(matches_df)
dups = []
for i in range(1000):
    for j in range(x):
        if News_df_s['text'][i] is matches_df['left_side'][j]:
            print(i,j)
            dups.append(i)
dups

14 1248
14 1249
45 1660
189 540
189 541
225 56
226 1251
559 605
561 1047
561 1048
648 1244
754 362
777 338
793 1275
866 1146
887 1128


[14, 14, 45, 189, 189, 225, 226, 559, 561, 561, 648, 754, 777, 793, 866, 887]

In [88]:
News_df_s['text'][59][:100]

'DHAKA (Reuters) - Bangladesh is to buy rice from Myanmar, putting aside worsening relations over the'

In [89]:
matches_df['left_side'][1005][:100]

'Following a Boston judge s refusal to renew the temporary stay on Trump s Muslim ban, a federal judg'

In [90]:
News_df_s['text'][79][:100]

"The video of Yulia Skripal's interview, published by Reuters in May 2018, is fake. It was filmed in "

In [91]:
matches_df['left_side'][500][:100]

'White House press secretary Sean Spicer literally denied the meaning of alleged president Donald Tru'

In [92]:
News_df_s['text'][436][:100]

'I certainly hope Hillary picks Liz Warren for VP! She s such a miserable woman who decided to attack'

In [93]:
matches_df['left_side'][1156][:100]

'Israeli Prime Minister Benjamin Netanyahu threw a temper tantrum like a petulant child because the U'

In [94]:
News_df_s['text'][489][:100]

'BOMBSHELL AUDIO: Hillary Clinton Herself Recorded Calling for Rigging Election \nPresident Obama clai'

In [95]:
#matches_df['left_side'][2087][:100]

In [96]:
News_df_s['text'][588][:100]

'An intense and sustained push by U.S. evangelicals helped drive President Donald Trump s decision to'

In [97]:
# This for loop seems to adequately find the indices of the duplicates lets do this and check on the full dataset. 
# 

In [98]:
x = len(matches_df)
y = len(News_df)
verify = []
dups = []
for i in range(y):
    for j in range(x):
        if News_df['text'][i] is matches_df['left_side'][j]:
            verify.append(j)
            dups.append(i)
dups

[3,
 7,
 8,
 19,
 21,
 27,
 27,
 28,
 35,
 54,
 59,
 59,
 70,
 71,
 71,
 75,
 78,
 82,
 82,
 88,
 88,
 95,
 96,
 98,
 98,
 100,
 103,
 109,
 113,
 114,
 114,
 118,
 120,
 121,
 126,
 143,
 144,
 148,
 159,
 161,
 165,
 166,
 168,
 170,
 177,
 180,
 187,
 189,
 192,
 192,
 193,
 195,
 198,
 203,
 205,
 210,
 215,
 216,
 217,
 220,
 221,
 224,
 227,
 228,
 232,
 244,
 245,
 249,
 251,
 257,
 257,
 259,
 264,
 281,
 297,
 305,
 305,
 307,
 313,
 317,
 323,
 323,
 327,
 338,
 338,
 338,
 345,
 345,
 350,
 364,
 377,
 380,
 380,
 382,
 386,
 392,
 394,
 394,
 400,
 402,
 408,
 409,
 415,
 423,
 423,
 430,
 438,
 447,
 454,
 458,
 458,
 465,
 472,
 476,
 481,
 486,
 488,
 489,
 494,
 496,
 499,
 505,
 507,
 509,
 512,
 515,
 522,
 534,
 536,
 544,
 551,
 553,
 560,
 561,
 562,
 572,
 573,
 584,
 604,
 605,
 610,
 616,
 623,
 623,
 623,
 623,
 623,
 623,
 623,
 625,
 628,
 630,
 630,
 631,
 636,
 642,
 643,
 645,
 648,
 651,
 652,
 655,
 660,
 664,
 664,
 666,
 668,
 669,
 670,
 671,
 674,
 6

In [99]:
len(dups)

2048

In [100]:
len(matches_df)

2048

In [101]:
len(verify)

2048

In [102]:
verify

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [103]:
#verify is literally just the i of dups I want to look at some of the duplicates in dups e.g. 623

In [104]:
dups.index(623)

142

In [105]:
#code adapted from https://datagy.io/python-list-find-all-index/
def find_indices(list_to_check, item_to_find):
    indices = []
    for idx, value in enumerate(list_to_check):
        if value == item_to_find:
            indices.append(idx)
    return indices

print(find_indices(dups,623))

[142, 143, 144, 145, 146, 147, 148]


In [106]:
News_df['text'][623][:100]

'The current occupant of the White House has been quiet about his campaign promise to combat the opio'

In [107]:
matches_df['left_side'][142][:100]

'The current occupant of the White House has been quiet about his campaign promise to combat the opio'

In [108]:
matches_df['left_side'][143][:100]

'The current occupant of the White House has been quiet about his campaign promise to combat the opio'

In [109]:
matches_df['left_side'][144][:100]

'The current occupant of the White House has been quiet about his campaign promise to combat the opio'

In [110]:
matches_df['left_side'][145][:100]

'The current occupant of the White House has been quiet about his campaign promise to combat the opio'

In [111]:
matches_df['left_side'][146][:100]

'The current occupant of the White House has been quiet about his campaign promise to combat the opio'

In [112]:
matches_df['left_side'][147][:100]

'The current occupant of the White House has been quiet about his campaign promise to combat the opio'

In [113]:
matches_df['left_side'][148][:100]

'The current occupant of the White House has been quiet about his campaign promise to combat the opio'

In [114]:
#Everything seems to checkout so I am going to go ahead and drop the found duplicates

In [115]:
dups = list(set(dups))
len(dups)

1344

In [116]:
News_df.drop(index = dups, inplace = True)

In [117]:
News_df

,text,real,subject,title,date
0,Donald Trump just couldn t wish all Americans ...,0,None,None,None
1,House Intelligence Committee Chairman Devin Nu...,0,None,None,None
2,"On Friday, it was revealed that former Milwauk...",0,None,None,None
4,Pope Francis used his annual Christmas Day mes...,0,None,None,None
5,The number of cases of cops brutalizing and ki...,0,None,None,None
...,...,...,...,...,...
130269,21st Century Wire says As 21WIRE reported earl...,0,Middle-east,McPain: John McCain Furious That Iran Treated ...,"January 16, 2016"
130270,21st Century Wire says It s a familiar theme. ...,0,Middle-east,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,"January 16, 2016"
130271,Patrick Henningsen 21st Century WireRemember ...,0,Middle-east,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,"January 15, 2016"
130272,21st Century Wire says Al Jazeera America will...,0,Middle-east,How to Blow $700 Million: Al Jazeera America F...,"January 14, 2016"


In [118]:
News_df.to_pickle('News_df.pkl')